In [1]:
from enum import Enum
from typing import Dict, Any, List, Optional, Union
from pydantic import BaseModel, Field, field_validator, ValidationError
from core import utils


#### /validators/validator.py

In [2]:
from validators.validator import TokenValidator

### Run 예시

In [3]:
import os
import logging
from core import logging_utils, utils

log_levels = {
    "DEBUG": logging.DEBUG,
    "INFO": logging.INFO,
    "WARNING": logging.WARNING,
    "ERROR": logging.ERROR,
    "CRITICAL": logging.CRITICAL
}

log_level = log_levels.get(os.environ.get("LOG_LEVEL", "INFO").upper(), logging.INFO)

logging_utils.setup_logging(
    log_dir="logs",
    log_level=log_level,
    max_bytes=10 * 1024 * 1024, 
    backup_count=5, 
    console_output=True 
)

logging_utils.cleanup_old_logs(log_dir="logs", days_to_keep=7)

17:50:50 - core.logging_utils - INFO - Logging initialized - logs directory: c:\Users\kakao\Desktop\AI_Agent\Semantic Layer\logs
17:50:50 - core.logging_utils - INFO - Log rotation configured: max 10.0MB per file, keeping 5 backups


In [4]:
resource_db = utils.load_resource_specs("./ResourceSpec/TokenSpec.yaml")

validator = TokenValidator(resource_db)

In [5]:
# --- 3. 검증 수행 (Runtime) ---

# Case A: 정상 토큰
valid_token = {"text": "Samsung Elec Risk Analysis...", "risk_score": 0.85}
validator.validate(valid_token, "RS_RISK_TOKEN_V1")

17:50:54 - Validator - INFO - 토큰 검증 시작 (Spec: RS_RISK_TOKEN_V1)


True

In [7]:
# Case B: 불량 토큰 (점수 초과)
invalid_token = {"text": "Short msg", "risk_score": 1.5} # Max 1.0 위반
validator.validate(invalid_token, "RS_RISK_TOKEN_V1")

17:51:36 - Validator - INFO - 토큰 검증 시작 (Spec: RS_RISK_TOKEN_V1)
17:51:36 - Validator - ERROR - 'risk_score' value 1.5 > max 1.0


SemanticError: 'risk_score' value 1.5 > max 1.0

In [48]:
# Case C: 불량 토큰 (필수값 누락)
missing_token = {"text": "No Score"}
validator.validate(missing_token, "RS_RISK_TOKEN_V1")

17:49:51 - validators.validator - INFO - [Validator] 토큰 검증 시작 (Spec: RS_RISK_TOKEN_V1)
17:49:51 - validators.validator - ERROR - Missing Required Field: 'risk_score'


True